In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import re
from espn_api.football import League

# Configure pandas for improved display
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)


C:\Users\andre\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
# ESPN API Setup
espn_s2 =  ESPN_S2
swid = SWID
league = League(league_id=1254749, year=2025, espn_s2=espn_s2, swid=swid)

# Fetch ESPN projections
def fetch_espn_projections():
    free_agents = league.free_agents()
    player_data = []
    for player in free_agents:
        player_entry = {
            "Player": player.name,
            "Player ID": player.playerId,
            "Position": player.position,
            "Current Team": player.proTeam,
            "Projected": player.projected_total_points
        }
        player_data.append(player_entry)
    return pd.DataFrame(player_data)

# Fetch and store ESPN data
espn_df = fetch_espn_projections()
print("ESPN Projections Loaded Successfully!")
display(espn_df.head())


ESPN Projections Loaded Successfully!


,Player,Player ID,Position,Current Team,Projected
0,Ja'Marr Chase,4362628,WR,CIN,339.01
1,Saquon Barkley,3929630,RB,PHI,325.10
2,Jahmyr Gibbs,4429795,RB,DET,317.12
3,Justin Jefferson,4262921,WR,MIN,324.48
4,Bijan Robinson,4430807,RB,ATL,335.82


In [3]:
def clean_player_name(name):
    """Cleans and standardizes a player's name for reliable merging."""
    if not isinstance(name, str):
        return None
    name = name.lower()
    name = name.replace('.', '').replace("'", "")
    name = re.sub(r'\s+(jr|sr|ii|iii|iv|v)$', '', name)
    return name.strip()

# Apply cleaning function
espn_df['Canonical_Name'] = espn_df['Player'].apply(clean_player_name)


In [4]:
# Load historical data
historical_files = {
    2024: 'FantasyData2024_clean.csv',
    2023: 'FantasyData2023_clean.csv',
    2022: 'FantasyData2022_clean.csv'
}

historical_dfs = []
for year, file_path in historical_files.items():
    try:
        df = pd.read_csv(file_path)
        df['Year'] = year
        df['Canonical_Name'] = df['Player'].apply(clean_player_name)
        historical_dfs.append(df)
    except FileNotFoundError:
        print(f"File {file_path} not found. Skipping...")

# Concatenate historical dataframes
if historical_dfs:
    historical_df = pd.concat(historical_dfs, ignore_index=True)
    print("Historical Data Loaded Successfully!")
else:
    print("No historical data loaded.")


Historical Data Loaded Successfully!


In [5]:
if 'historical_df' in locals():
    master_df = pd.merge(espn_df, historical_df, on='Canonical_Name', how='left')
    master_df.rename(columns={'Player_x': 'Player', 'Player_y': 'Historical_Player'})[['Player', 'Player ID', 'Position', 'Current Team', 'Projected']]
    master_df.drop(columns=['Player_y'], inplace=True, errors='ignore')
    print("Master DataFrame Created!")
    display(master_df.head())


Master DataFrame Created!


,Player_x,Player ID,Position,Current Team,Projected,Canonical_Name,Rk,Tm,FantPos,Age,G,GS,Pass_Cmp,Pass_Att,Pass_Yds,Pass_TD,Pass_Int,Rush_Att,Rush_Yds,Y/A,Rush_TD,Tgt,Rec,Yds,Y/R,Receiving_TD,Fmb,FL,Total_TD,2:00 PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank,Player_Touches,Player Yards,Team Tuches,Team Yard,Team TD,% of Team Touch,% of Team Yard,% of Team TD,Year,0.583333333
0,Ja'Marr Chase,4362628,WR,CIN,339.01,jamarr chase,5.0,CIN,WR,24.0,17.0,16.0,0.0,0.0,0.0,0.0,0.0,3.0,32.0,10.67,0.0,175.0,127.0,1708.0,13.45,17.0,0.0,0.0,17.0,NaN,NaN,276.0,403.0,406.0,339.5,138.0,1.0,5.0,178.0,1740.0,926.0,6156.0,54.0,0.190000,0.280000,0.310000,2024.0,NaN
1,Ja'Marr Chase,4362628,WR,CIN,339.01,jamarr chase,33.0,CIN,WR,23.0,16.0,16.0,1.0,1.0,-7.0,0.0,0.0,3.0,-6.0,-2.00,0.0,145.0,100.0,1216.0,12.16,7.0,1.0,0.0,7.0,NaN,NaN,163.0,262.7,265.7,212.7,31.0,13.0,33.0,148.0,1210.0,914.0,5574.0,40.0,0.160000,0.220000,0.180000,2023.0,NaN
2,Ja'Marr Chase,4362628,WR,CIN,339.01,jamarr chase,56.0,CIN,WR,22.0,12.0,12.0,0.0,0.0,0.0,0.0,0.0,5.0,8.0,1.60,0.0,134.0,87.0,1046.0,12.02,9.0,2.0,2.0,9.0,NaN,NaN,155.0,242.4,247.4,198.9,38.0,12.0,32.0,139.0,1054.0,912.0,5794.0,49.0,0.152412,0.181912,0.183673,2022.0,NaN
3,Saquon Barkley,3929630,RB,PHI,325.10,saquon barkley,1.0,PHI,RB,27.0,16.0,16.0,0.0,0.0,0.0,0.0,0.0,345.0,2005.0,5.81,13.0,43.0,33.0,278.0,8.42,2.0,2.0,1.0,15.0,3.0,NaN,322.0,355.3,362.3,338.8,163.0,1.0,1.0,388.0,2283.0,871.0,5911.0,53.0,0.450000,0.390000,0.280000,2024.0,NaN
4,Saquon Barkley,3929630,RB,PHI,325.10,saquon barkley,43.0,NYG,RB,26.0,14.0,14.0,0.0,0.0,0.0,0.0,0.0,247.0,962.0,3.89,6.0,60.0,41.0,280.0,6.83,4.0,2.0,2.0,10.0,1.0,NaN,182.0,223.2,231.2,202.7,23.0,13.0,43.0,307.0,1242.0,823.0,4625.0,25.0,0.370000,0.270000,0.400000,2023.0,NaN


In [7]:
# Define key stats to analyze
stats_to_pivot = [
    'G', 'GS', 'PPR', 'Tgt', 'Rec', 'Yds', 
    'Rush_Att', 'Rush_Yds', 'FantPt',
    '% of Team Touch', '% of Team Yard', '% of Team TD'
]

# Create pivoted dataframe
pivoted_df = master_df.pivot_table(
    index=['Canonical_Name', 'Player ID', 'Position', 'Current Team', 'Projected'],
    columns='Year',
    values=stats_to_pivot
)

# Flatten column names
pivoted_df.columns = [f'{stat}_{int(year)}' for stat, year in pivoted_df.columns]
pivoted_df.reset_index(inplace=True)
print("Pivoted DataFrame Ready!")
display(pivoted_df.head())


Pivoted DataFrame Ready!


,Canonical_Name,Player ID,Position,Current Team,Projected,% of Team TD_2022,% of Team TD_2023,% of Team TD_2024,% of Team Touch_2022,% of Team Touch_2023,% of Team Touch_2024,% of Team Yard_2022,% of Team Yard_2023,% of Team Yard_2024,FantPt_2022,FantPt_2023,FantPt_2024,G_2022,G_2023,G_2024,GS_2022,GS_2023,GS_2024,PPR_2022,PPR_2023,PPR_2024,Rec_2022,Rec_2023,Rec_2024,Rush_Att_2022,Rush_Att_2023,Rush_Att_2024,Rush_Yds_2022,Rush_Yds_2023,Rush_Yds_2024,Tgt_2022,Tgt_2023,Tgt_2024,Yds_2022,Yds_2023,Yds_2024
0,aj brown,4047646,WR,PHI,275.48,0.192982,0.15,0.13,0.165525,0.18,0.11,0.244845,0.26,0.18,212.0,184.0,150.0,17.0,17.0,13.0,16.0,17.0,13.0,299.6,289.6,216.9,88.0,106.0,67.0,0.0,0.0,0.0,0.0,0.0,0.0,145.0,158.0,97.0,1496.0,1456.0,1079.0
1,alvin kamara,3054850,RB,NO,264.73,0.114286,0.15,0.24,0.336323,0.27,0.36,0.241553,0.20,0.30,155.0,158.0,197.0,15.0,13.0,14.0,13.0,12.0,14.0,211.7,233.0,265.3,57.0,75.0,68.0,223.0,180.0,228.0,897.0,694.0,950.0,77.0,86.0,89.0,490.0,466.0,543.0
2,amon-ra st brown,4374302,WR,DET,290.33,0.120000,0.18,0.18,0.161964,0.16,0.14,0.206545,0.23,0.18,162.0,212.0,201.0,16.0,16.0,17.0,16.0,16.0,17.0,267.6,330.9,316.2,106.0,119.0,115.0,9.0,4.0,2.0,95.0,24.0,6.0,146.0,164.0,141.0,1161.0,1515.0,1263.0
3,bijan robinson,4430807,RB,ATL,335.82,NaN,0.26,0.38,NaN,0.32,0.37,NaN,0.27,0.29,NaN,188.0,281.0,NaN,17.0,17.0,NaN,16.0,17.0,NaN,246.3,341.7,NaN,58.0,61.0,NaN,214.0,304.0,NaN,976.0,1456.0,NaN,86.0,72.0,NaN,487.0,431.0
4,brian thomas,4432773,WR,JAX,270.75,NaN,NaN,0.30,NaN,NaN,0.16,NaN,NaN,0.25,NaN,NaN,197.0,NaN,NaN,17.0,NaN,NaN,16.0,NaN,NaN,284.0,NaN,NaN,87.0,NaN,NaN,6.0,NaN,NaN,48.0,NaN,NaN,133.0,NaN,NaN,1282.0


In [9]:
# Calculate PPR changes
pivoted_df['PPR_Change_24vs23'] = pivoted_df['PPR_2024'] - pivoted_df['PPR_2023']
pivoted_df['PPR_Change_23vs22'] = pivoted_df['PPR_2023'] - pivoted_df['PPR_2022']

# Identify 3-year and 2-year risers
pivoted_df['is_3yr_riser'] = (pivoted_df['PPR_2024'] > pivoted_df['PPR_2023']) & \
                              (pivoted_df['PPR_2023'] > pivoted_df['PPR_2022']) & \
                              (pivoted_df['PPR_2022'] > 0)
pivoted_df['is_2yr_riser'] = (pivoted_df['PPR_2024'] > pivoted_df['PPR_2023']) & \
                             (pivoted_df['PPR_2023'] >= 0)

# Display top gainers
top_3yr_risers = pivoted_df[pivoted_df['is_3yr_riser']].sort_values('PPR_2024', ascending=False)
print("Top 3-Year PPR Gainers:")
display(top_3yr_risers[['Canonical_Name', 'Position', 'PPR_2022', 'PPR_2023', 'PPR_2024']].head(30))


Top 3-Year PPR Gainers:


,Canonical_Name,Position,PPR_2022,PPR_2023,PPR_2024
31,lamar jackson,QB,236.1,331.2,430.4
19,jamarr chase,WR,242.4,262.7,403.0
29,kyren williams,RB,30.5,255.0,272.1
20,james cook,RB,105.7,232.7,266.7
1,alvin kamara,RB,211.7,233.0,265.3
41,trey mcbride,TE,61.5,181.5,249.8
24,jonathan taylor,RB,146.4,156.4,244.7
16,george kittle,TE,200.5,203.2,236.6
